**Практическая работа #1**

1. hit rate at k

2. money_precision_at_k

3. recall_at_k
4. money_recall_at_k

5. map@k
6. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции

7. Mean Reciprocal Rank
8. *average_precision для ранжирования по примеру с картинки лекции



In [27]:
import numpy as np

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [6]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
np.random.seed(0)
prices_recommended = np.random.randint(1e3, size=len(recommended_list))

prices_bought = [prices_recommended[i] if (recommended_list[i] in bought_list) else np.random.randint(1e3) for i in range(len(bought_list))] 

def hit_rate_at_k(recommended_list, bought_list, k=5):
    """
    Returns 1 if there was at least one relevant product 
    among the k (default 5) recommended products
    """
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    print('flags:\t\t\t{}'.format(flags))

    hit_rate = (flags.sum() > 0) * 1

    return hit_rate
    
print('recommended_list:\t{}'.format(recommended_list))
print('bought_list:\t\t{}'.format(bought_list))
hr = hit_rate_at_k(recommended_list, bought_list)
print('hit rate:\t\t{}'.format(hr))

recommended_list:	[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list:		[521, 32, 143, 991]
flags:			[False False  True  True]
hit rate:		1


### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  


In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    """Precision measures how accurate is our predictions."""
    recommended_list = recommended_list[:k]
        
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list) 

    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    relevant_revenue = sum(prices_recommended[np.isin(recommended_list, bought_list)])
    recommend_revenue = sum(prices_recommended)

    precision = relevant_revenue / recommend_revenue
    #info
    print("rec_items:\t\t{}".format(recommended_list), 
          "\nrec_prices:\t\t{} (sum = {})".format(prices_recommended, recommend_revenue),
          "\nbought_items:\t\t{}".format(bought_list))
    print("relevant_revenue:\t{}".format(relevant_revenue))

    return precision
k=5
mp = money_precision_at_k(recommended_list, bought_list, prices_recommended, k)
print("money precision@{}:\t{}".format(
    k, mp
    ))

print('\nПользователь потратил около %.f' % (mp*100) \
     +f'% от суммы стоимостей топ-{k} рекомендованных товаров' )

rec_items:		[ 143  156 1134  991   27] 
rec_prices:		[684 559 629 192 835] (sum = 2899) 
bought_items:		[521  32 143 991]
relevant_revenue:	876
money precision@5:	0.30217316315971027

Пользователь потратил около 30% от суммы стоимостей топ-5 рекомендованных товаров


### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

In [8]:
def recall_at_k(recommended_list, bought_list, k=5):
    """Recall measures how good you find all the positives."""
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
 
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):

    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    # revenue of relevant recommended items 
    relevant_revenue = sum(prices_recommended[np.isin(recommended_list, bought_list)])
    # revenue of relevant items
    total_revenue = sum(prices_bought)
    recall = relevant_revenue / total_revenue
    
    # info
    print("recommended_list @{}:\t{}".format(k, recommended_list))
    print("prices_recommended @{}:\t{}".format(k, prices_recommended))
    print("bought_list:\t\t{}".format(bought_list))
    print("prices_bought:\t\t{} (sum = {})".format(prices_bought, total_revenue))
    print("relevant_revenue:\t{}".format(relevant_revenue))
    print('money_recall @{}:\t{}'.format(k, recall))
    return recall

mr = money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)
print("\nОколо %.f" % (mr*100)+'% дохода от пользователя было предсказано рекомендацией.')

recommended_list @5:	[ 143  156 1134  991   27]
prices_recommended @5:	[684 559 629 192 835]
bought_list:		[521, 32, 143, 991]
prices_bought:		[684 277 754 192] (sum = 1907)
relevant_revenue:	876
money_recall @5:	0.4593602517042475

Около 46% дохода от пользователя было предсказано рекомендацией.


## AP@k
AP@k - average precision at k
$$Average\,Precision = {{\sum\nolimits _{r}P @ r}\over {R}}$$
- r - кол-во релевантных среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [9]:
def average_precision_at_k(bought_list, recommended_list, k=5):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    bought_list : list
            A list of elements that are to be recommended (order doesn't matter)
    recommended_list : list
            A list of recommended elements (order does matter)
    k : int, optional
            Select k top elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(recommended_list)>k:
        recommended_list = recommended_list[:k]
    print('bought_list:', bought_list)
    print(f'recommender_list@{k}:', recommended_list)
    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(recommended_list):
        if p in bought_list:
            num_hits += 1.0
            score += precision_at_k(recommended_list[:i+1], bought_list, i+1)
    if not bought_list:
        return 0.0
    return score / len(bought_list)


print(f'AP@{k} =', average_precision_at_k(bought_list, recommended_list, k))

bought_list: [521, 32, 143, 991]
recommender_list@5: [143, 156, 1134, 991, 27]
AP@5 = 0.375


## MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров


In [10]:
# Добавлю еще пару пользователей с одинаковыми рекомендациями для этой задачи
actual = [[521, 32, 143, 991], [432, 479, 1134, 156, 43], [34, 156, 43, 1134]]
recs = [recommended_list]*3

def map_k(recommended_list, bought_list, k):
    
    n_users = len(bought_list) 
    print('n_users:', n_users)
    sum_ = 0
    for u in range(n_users):
        print(f'\n{u+1}')
        ap_k = average_precision_at_k(bought_list[u], recommended_list[u], k)
        sum_ += ap_k
        print(f'apk: {ap_k}')
    result = sum_ / n_users
    
    return result
k=5
print(f'------\nMAP@{k} =', map_k(recs, actual, k))


n_users: 3

1
bought_list: [521, 32, 143, 991]
recommender_list@5: [143, 156, 1134, 991, 27]
apk: 0.375

2
bought_list: [432, 479, 1134, 156, 43]
recommender_list@5: [143, 156, 1134, 991, 27]
apk: 0.2333333333333333

3
bought_list: [34, 156, 43, 1134]
recommender_list@5: [143, 156, 1134, 991, 27]
apk: 0.29166666666666663
------
MAP@5 = 0.3


## NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


<p style='color:red'>(!) Считаем для первых k рекоммендаций   </p> 
<p style='color:red'>(!) - существуют вариации с другими $discount(i)$</p>  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 



$NDCG = \frac{DCG}{ideal DCG}$

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [11]:
def discount(flag, i):
    if i <= 2:
        return flag / i
    else:
        return flag / np.log2(i)


def dcg_at_k(recommended_list, bought_list, k=5):
    """
  Discounted Cumulative Gain
  ------
   Popular measure for evaluating web search and
        related tasks
   Two assumptions:
      - Highly relevant documents are more useful
        than marginally relevant document
      - The lower the ranked position of a relevant
        document, the less useful it is for the user,
        since it is less likely to be examined
    
    Gain is accumulated starting at the top of the
        ranking and may be reduced, or discounted, at lower ranks.
        Typical discount is 1/log (rank)
    """
    bought_list = np.array(bought_list)
    recommended_list = recommended_list[:k]
    rec_len = len(recommended_list)

    bool_mask = np.isin(recommended_list, bought_list)
    flags = bool_mask * 1

    sum_ = 0
    for i in range(1, rec_len+1):
        sum_ += discount(flags[i-1], i)
    dcg_score = sum_ / rec_len

    return dcg_score


def ideal_dcg(rec_len):
    flags = np.ones(rec_len)
    sum_ = 0
    for i in range(1, rec_len+1):
        sum_ += discount(flags[i-1], i)
    ideal_dcg = sum_ / rec_len

    return ideal_dcg


def ndcg(recommended_list, bought_list, k=5):
    dcg_score = dcg_at_k(recommended_list, bought_list, k)

    if k:
        rec_len = k   
    else:
        rec_len = len(recommended_list)
    
    return dcg_score / ideal_dcg(rec_len)


print("DCG Score =", dcg_at_k(recommended_list, bought_list, k=None))
print("NDCG Score =", ndcg(recommended_list, bought_list, k=None))
print('-------')
print("DCG Score@5 =", dcg_at_k(recommended_list, bought_list))
print("NDCG Score@5 =", ndcg(recommended_list, bought_list))

DCG Score = 0.15
NDCG Score = 0.31549095204264654
-------
DCG Score@5 = 0.3
NDCG Score@5 = 0.489938890671454


## MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [26]:
def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    ranks = []
    for u in range(len(bought_list)):
        for c in bought_list[u]:

            if c in recommended_list[u]:
                print('recs:', recommended_list[u])
                print('actual:', bought_list[u])
                ranks.append(list(recommended_list[u]).index(c)+1)
                print([ranks[u]], '\n')
                break
    
    reciprocal_ranks = list(map(lambda x: x**(-1), ranks))
    print('reciprocal_ranks:', reciprocal_ranks)
    mrr = np.mean(reciprocal_ranks)

    return mrr

mrr = mean_reciprocal_rank(recs, actual)
print("------\nMean Reciprocal Rank@5 =", mrr)

recs: [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
actual: [521, 32, 143, 991]
[1] 

recs: [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
actual: [432, 479, 1134, 156, 43]
[3] 

recs: [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
actual: [34, 156, 43, 1134]
[2] 

reciprocal_ranks: [1.0, 0.3333333333333333, 0.5]
------
Mean Reciprocal Rank@5 = 0.611111111111111
